In [16]:
import requests as req
import os
import json
import re  # 匯入正規表達式模組，用於清洗檔名
from bs4 import BeautifulSoup as bs

def sanitize_filename(name):
    """
    清洗檔名，移除 Windows/Linux 檔案系統不允許的字元
    """
    # 將不合法的字元替換為底線
    return re.sub(r'[\\/*?:"<>|]', '_', name).strip()

def download_stickers(url):
    """
    主程式：傳入 LINE 貼圖網址，自動建立資料夾並下載
    """
    # 設定 User-Agent 模擬瀏覽器，避免被擋
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    print(f"正在分析頁面: {url} ...")
    
    try:
        res = req.get(url, headers=headers)
        res.raise_for_status() # 檢查請求是否成功
    except Exception as e:
        print(f"無法讀取網頁: {e}")
        return

    soup = bs(res.text, 'lxml')

    # 1. 抓取貼圖標題 (作為資料夾名稱)
    # LINE Store 的標題通常在 p.mdCMN38Item01Ttl 或 h3.mdCMN08Ttl 內
    title_element = soup.select_one("p.mdCMN38Item01Ttl") or soup.select_one("h3.mdCMN08Ttl")
    
    if title_element:
        folder_name = sanitize_filename(title_element.text)
    else:
        # 若抓不到標題，用貼圖 ID 或預設名稱
        folder_name = f"unknown_sticker_{url.split('/')[-2]}"

    # 設定主目錄與子目錄
    base_folder = 'line_stickers'
    target_folder = os.path.join(base_folder, folder_name)

    # 建立資料夾 (如果父目錄不存在也會一併建立)
    if not os.path.exists(target_folder):
        os.makedirs(target_folder)
        print(f"已建立資料夾: {target_folder}")
    else:
        print(f"資料夾已存在: {target_folder}")

    # 2. 取得貼圖資訊
    li_elements = soup.select("ul.mdCMN09Ul.FnStickerList > li.mdCMN09Li.FnStickerPreviewItem")
    
    print(f"找到 {len(li_elements)} 張貼圖，準備下載...")

    # 3. 下載圖片 (使用 Python requests 取代 curl)
    success_count = 0
    
    for li in li_elements:
        try:
            strJson = li.get("data-preview")
            obj = json.loads(strJson)
            
            id_ = obj['id']
            link = obj['staticUrl']
            
            # 設定存檔路徑
            file_path = os.path.join(target_folder, f"{id_}.png")
            
            # 下載圖片內容
            img_res = req.get(link, stream=True)
            if img_res.status_code == 200:
                with open(file_path, 'wb') as f:
                    for chunk in img_res.iter_content(1024):
                        f.write(chunk)
                success_count += 1
                # 簡單進度條效果
                print(f"\r下載進度: [{success_count}/{len(li_elements)}] ID: {id_}", end="")
            else:
                print(f"\n下載失敗 ID: {id_}, Status: {img_res.status_code}")
                
        except Exception as e:
            print(f"\n處理貼圖 ID {obj.get('id', 'unknown')} 時發生錯誤: {e}")

    print(f"\n\n下載完成！\n貼圖已儲存於: {os.path.abspath(target_folder)}")

if __name__ == "__main__":
    # 您可以在這裡更換想要的貼圖網址
    sticker_url = 'https://store.line.me/stickershop/product/31862075/zh-Hant?from=sticker'
    
    download_stickers(sticker_url)

正在分析頁面: https://store.line.me/stickershop/product/31862075/zh-Hant?from=sticker ...
已建立資料夾: line_stickers\工作犬求生指南2：秋意濃
找到 40 張貼圖，準備下載...
下載進度: [40/40] ID: 795131832

下載完成！
貼圖已儲存於: c:\Users\user\Documents\GitHub\TAX-COMPUTE\line_stickers\工作犬求生指南2：秋意濃
